### Video Classification using only LSTM

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp 

In [2]:
mp_holist = mp.solutions.holistic 
mp_draw = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False                 
    result = model.process(img)                 # Make prediction
    img.flags.writeable = True                   
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
    return img, result

In [4]:
def draw_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS) # Draw face connections
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS) # Draw pose connections
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw left hand connections
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
                             mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
                             mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
                             ) 
    # mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
    #                          mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
    #                          mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
    #                          ) 
    
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS,
                             mp_draw.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
mp_holist.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


C:\Users\araya\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [8]:
cap.release()
cv2.destroyAllWindows()

In [9]:
results.pose_landmarks.landmark[0].visibility

0.9963851571083069

In [10]:
len(results.pose_landmarks.landmark)

33

In [11]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*2)
    left_hnd=np.array([[res.x,res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
    right_hnd=np.array([[res.x,res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
    face=np.array([[res.x,res.y] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*2)
    return np.concatenate([pose,left_hnd,right_hnd,face])
# concatenating for the model to detect the sign language

In [12]:
extract_keypoints(results).shape

(1086,)

In [13]:
import os 
video_dir = "C:/Users/araya/Desktop/keypoints/video_extract"
video_list = []
video_list = os.listdir(video_dir)

len(video_list)

160

In [14]:
video_list

['กฎกระทรวง.mp4',
 'กฎกระทรวง_0.mp4',
 'กฎกระทรวง_1.mp4',
 'กฎกระทรวง_2.mp4',
 'กฎหมายรัฐธรรมนูญ.mp4',
 'กฎหมายรัฐธรรมนูญ_0.mp4',
 'กฎหมายรัฐธรรมนูญ_1.mp4',
 'กฎหมายรัฐธรรมนูญ_2.mp4',
 'กรมอนามัย.mp4',
 'กรมอนามัย_0.mp4',
 'กรมอนามัย_1.mp4',
 'กรมอนามัย_2.mp4',
 'กรรม.mp4',
 'กรรม_0.mp4',
 'กรรม_1.mp4',
 'กรรม_2.mp4',
 'กรรมสิทธิ์.mp4',
 'กรรมสิทธิ์_0.mp4',
 'กรรมสิทธิ์_1.mp4',
 'กรรมสิทธิ์_2.mp4',
 'กระโดด.mp4',
 'กระโดด_0.mp4',
 'กระโดด_1.mp4',
 'กระโดด_2.mp4',
 'กล้วยบวชชี.mp4',
 'กล้วยบวชชี_0.mp4',
 'กล้วยบวชชี_1.mp4',
 'กล้วยบวชชี_2.mp4',
 'กล้วยเชื่อม.mp4',
 'กล้วยเชื่อม_0.mp4',
 'กล้วยเชื่อม_1.mp4',
 'กล้วยเชื่อม_2.mp4',
 'กังวล.mp4',
 'กังวล_0.mp4',
 'กังวล_1.mp4',
 'กังวล_2.mp4',
 'กีฬา.mp4',
 'กีฬา_0.mp4',
 'กีฬา_1.mp4',
 'กีฬา_2.mp4',
 'น้อง.mp4',
 'น้อง_0.mp4',
 'น้อง_1.mp4',
 'น้อง_2.mp4',
 'เขิน.mp4',
 'เขิน_0.mp4',
 'เขิน_1.mp4',
 'เขิน_2.mp4',
 'เขื่อนดิน.mp4',
 'เขื่อนดิน_0.mp4',
 'เขื่อนดิน_1.mp4',
 'เขื่อนดิน_2.mp4',
 'เขื่อนสิริกิติ์.mp4',
 'เขื่อนสิริกิติ์_0.mp4',


In [15]:
# Path for exported data, numpy arrays
Model_Data=os.path.join('Data for different actions')

actions = np.array(video_list)

no_of_seqs = 1

# 30 frames in length
seq_length = 210

In [16]:
actions

array(['กฎกระทรวง.mp4', 'กฎกระทรวง_0.mp4', 'กฎกระทรวง_1.mp4',
       'กฎกระทรวง_2.mp4', 'กฎหมายรัฐธรรมนูญ.mp4',
       'กฎหมายรัฐธรรมนูญ_0.mp4', 'กฎหมายรัฐธรรมนูญ_1.mp4',
       'กฎหมายรัฐธรรมนูญ_2.mp4', 'กรมอนามัย.mp4', 'กรมอนามัย_0.mp4',
       'กรมอนามัย_1.mp4', 'กรมอนามัย_2.mp4', 'กรรม.mp4', 'กรรม_0.mp4',
       'กรรม_1.mp4', 'กรรม_2.mp4', 'กรรมสิทธิ์.mp4', 'กรรมสิทธิ์_0.mp4',
       'กรรมสิทธิ์_1.mp4', 'กรรมสิทธิ์_2.mp4', 'กระโดด.mp4',
       'กระโดด_0.mp4', 'กระโดด_1.mp4', 'กระโดด_2.mp4', 'กล้วยบวชชี.mp4',
       'กล้วยบวชชี_0.mp4', 'กล้วยบวชชี_1.mp4', 'กล้วยบวชชี_2.mp4',
       'กล้วยเชื่อม.mp4', 'กล้วยเชื่อม_0.mp4', 'กล้วยเชื่อม_1.mp4',
       'กล้วยเชื่อม_2.mp4', 'กังวล.mp4', 'กังวล_0.mp4', 'กังวล_1.mp4',
       'กังวล_2.mp4', 'กีฬา.mp4', 'กีฬา_0.mp4', 'กีฬา_1.mp4',
       'กีฬา_2.mp4', 'น้อง.mp4', 'น้อง_0.mp4', 'น้อง_1.mp4', 'น้อง_2.mp4',
       'เขิน.mp4', 'เขิน_0.mp4', 'เขิน_1.mp4', 'เขิน_2.mp4',
       'เขื่อนดิน.mp4', 'เขื่อนดิน_0.mp4', 'เขื่อนดิน_1.mp4',
       'เขื่อนดิ

In [17]:
# just creating the folders and sub folders

for action in actions: 
    try: 
        os.makedirs(os.path.join(Model_Data, action))
    except:
        pass

In [18]:
# import os
# augment_dir = "C:/Users/araya/Desktop/augments"

# augment_list = []
# augment_list = os.listdir(augment_dir)
# augment_list

In [19]:
# actions = list(actions)

In [20]:
# for x in augment_list:
#     # print(x)
#     actions.append(x)
# actions

In [21]:
# actions = np.array(actions)
# actions

Collecting keypoint values for Training nd Testing

In [22]:
# Define the directory where your videos are stored
directory = "C:/Users/araya/Desktop/keypoints/video_extract"

In [23]:
directory

'C:/Users/araya/Desktop/keypoints/video_extract'

In [24]:
for filename in actions:
    print(directory + '/' + filename)

C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง_0.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง_1.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง_2.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎหมายรัฐธรรมนูญ.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎหมายรัฐธรรมนูญ_0.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎหมายรัฐธรรมนูญ_1.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎหมายรัฐธรรมนูญ_2.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรมอนามัย.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรมอนามัย_0.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรมอนามัย_1.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรมอนามัย_2.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรรม.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรรม_0.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรรม_1.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรร

In [25]:
# # Set mediapipe model 
# for action in actions:
#     video_path = os.path.join("C:/Users/araya/Desktop/keypoints/video_extract", action)
#     cap = cv2.VideoCapture(video_path)
#     cap.set(cv2.CAP_PROP_FPS, 60)
#     length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("LENGTH:" + str(length))
#     # keypoints = []

#     if not cap.isOpened():
#         print(f"Error opening video file: {video_path}")
#         continue

#     with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         for seq in range(no_of_seqs):
#             for frame_num in range(seq_length):

#                 ret, frame = cap.read()
#                 if not ret:
#                     print(f"End of video {video_path}")
#                     break
                
#                 img, results = mediapipe_detection(frame, holistic)
#                 draw_styled_landmarks(img, results)

#                 # print(frame_num)

#                 if frame_num == 0: 
#                     cv2.putText(img, 'DATA COLLECTION STARTED', (120,200), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Window', img)
#                     cv2.waitKey(2000)  # 2 seconds delay for setup
#                 else: 
#                     cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Window', img)

#                 keypoints = extract_keypoints(results)
#                 # keypoints.append(results)
#                 npy_path = os.path.join(Model_Data, action, f"frame_{frame_num}.npy")
#                 os.makedirs(os.path.dirname(npy_path), exist_ok=True)
#                 np.save(npy_path, keypoints)

#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     break

#             if not ret:
#                 break

#     cap.release()
#     cv2.destroyAllWindows()

In [26]:
# Set mediapipe model 
for action in actions:
    video_path = os.path.join("C:/Users/araya/Desktop/keypoints/video_extract", action)
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_FPS, 60)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("LENGTH:" + str(length))
    keypoints = []

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        continue

    with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for seq in range(no_of_seqs):
            for frame_num in range(seq_length):

                ret, frame = cap.read()
                if not ret:
                    print(f"End of video {video_path}")
                    break
                
                img, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(img, results)

                # print(frame_num)

                if frame_num == 0: 
                    cv2.putText(img, 'DATA COLLECTION STARTED', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Window', img)
                    cv2.waitKey(2000)  # 2 seconds delay for setup
                else: 
                    cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Window', img)

                x = extract_keypoints(results)
                keypoints.append(x)
                npy_path = os.path.join(Model_Data, action, f"{action.split(".")[0]}.npy")
                os.makedirs(os.path.dirname(npy_path), exist_ok=True)
                np.save(npy_path, keypoints)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if not ret:
                break

    cap.release()
    cv2.destroyAllWindows()

LENGTH:142
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎกระทรวง.mp4
LENGTH:142
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎกระทรวง_0.mp4
LENGTH:142
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎกระทรวง_1.mp4
LENGTH:142
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎกระทรวง_2.mp4
LENGTH:134
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎหมายรัฐธรรมนูญ.mp4
LENGTH:134
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎหมายรัฐธรรมนูญ_0.mp4
LENGTH:134
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎหมายรัฐธรรมนูญ_1.mp4
LENGTH:134
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎหมายรัฐธรรมนูญ_2.mp4
LENGTH:151
End of video C:/Users/araya/Desktop/keypoints/video_extract\กรมอนามัย.mp4
LENGTH:151
End of video C:/Users/araya/Desktop/keypoints/video_extract\กรมอนามัย_0.mp4
LENGTH:151
End of video C:/Users/araya/Desktop/keypoints/video_extract\กรมอนามัย_1.mp4
LENGTH:151
End of vid

In [27]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
file_paths = []
for action in actions:
    video_path = os.path.join('Data for different actions/', action)
    # print(video_path)
    # print(action)
    file_paths.append(video_path + '/' + action.split(".")[0] + ".npy")
print(file_paths)

In [29]:
def load_keypoint_sequences(file_paths):
    keypoint_sequences = []
    for file_path in file_paths:
        keypoints = np.load(file_path)
        keypoint_sequences.append(torch.tensor(keypoints, dtype=torch.float32))
    return keypoint_sequences

In [ ]:
# Load the sequences
import torch
sequences = load_keypoint_sequences(file_paths)
sequences

In [ ]:
# Pad the sequences to the same length
from torch.nn.utils.rnn import pad_sequence
padded_sequences = pad_sequence(sequences, batch_first=True)
pad_sequence
print(padded_sequences.shape) # (batch_size, max_sequence_length, num_keypoints)

In [ ]:
labels = [action.split(".")[0] for action in actions]
labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

labels = le.fit_transform(labels)
labels

In [34]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# Create a custom dataset
class KeypointDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        keypoints = np.load(self.file_paths[idx])
        label = self.labels[idx]
        return torch.tensor(keypoints, dtype=torch.float32), label

In [35]:
# Create the dataset
dataset = KeypointDataset(file_paths, labels)

In [ ]:
print(dataset.file_paths)
print(dataset.labels)

In [37]:
# Collate function for padding
def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_sequences = pad_sequence(sequences, batch_first=True)
    return padded_sequences, torch.tensor(labels)

In [ ]:
# Create the DataLoader
batch_size = 4
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate the LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Use the last time step's output for classification
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
# Set device (use GPU if available)
device = torch.device('cpu')
device

In [41]:
# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size=1662, hidden_size=128, num_layers=2, num_classes=40).to(device)

In [42]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
# References : https://saturncloud.io/blog/calculating-the-accuracy-of-pytorch-models-every-epoch/#:~:text=In%20order%20to%20calculate%20the,tensor%20along%20a%20specified%20dimension
num_epochs = 1200
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0
    model.train()
    for i, (sequences, labels) in enumerate(data_loader):
        # Move data to the device
        sequences = sequences.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sequences)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    accuracy = 100 * total_correct /total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} , Accuracy : {accuracy:.2f}%')

### Save and Load Model

In [ ]:
pad_sequence(sequences, batch_first=True)

In [ ]:
# Put the model in evaluation mode
model.eval()

# No need to track gradients during inference
with torch.no_grad():
    # Get the model's output (logits)
    outputs = model(padded_sequences)

# outputs = torch.softmax(outputs, dim=1)
# outputs = torch.max(outputs,1)

outputs


In [ ]:
padded_sequences

In [729]:
file_paths = ["Data for different actions/ไอซ์.mp4/ไอซ์.npy"]

In [ ]:
# Load the sequences
import torch
sequences = load_keypoint_sequences(file_paths)
# Change list to numpy array 
sequences = np.array(sequences)
# Change numpy array to tensor
sequences = torch.FloatTensor(sequences)
sequences = pad_sequence(sequences, batch_first=True)
sequences

In [ ]:
outputs = model(sequences)
outputs

In [ ]:
labels = [action.split(".")[0] for action in actions]
labels

In [733]:
# Change from tensor to numpy arrat
outputs = outputs.detach().numpy()

In [ ]:
outputs

In [ ]:
for idx, word in enumerate(outputs):
    # max_value = torch.max(outputs)
    list_outputs = max(outputs)
    print(list_outputs)
    # print(max_value)
    # print(max_value.detach().numpy())

In [ ]:
index_max = max(range(len(list_outputs)), key=list_outputs.__getitem__)
index_max

In [ ]:
print(labels[index_max])

### -------------------------------------------------------------------------------------------------------------------------------------------- ###

In [ ]:
model.eval()

In [740]:
def check_accuracy(loader, model):
    if data_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")  
          
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
        
    model.train()

In [ ]:
check_accuracy(data_loader, model)